In [1]:
import os

In [2]:
import os
import polars as pl
import pyspark 
from pyspark.sql.functions import col

ModuleNotFoundError: No module named 'polars'

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("EDA") \
    .getOrCreate()

In [ ]:
"data/raw/amex-default-prediction/train_labels.csv"

In [ ]:
label_df = spark.read.option("header", "true").csv(
    "/workspace/CreditRiskAI/data/raw/amex-default-prediction/train_labels.csv"
)

In [ ]:
label_df.show(10)

In [ ]:
df_full = spark.read.option("header", "true").option("nullValue", "?").\
    option("inferSchmea", "true").csv("/workspace/CreditRiskAI/data/raw/amex-default-prediction/train_data.csv")

In [ ]:
df_full.show(10)

In [ ]:
label_df.printSchema()

In [ ]:
df_full.printSchema()

In [ ]:
example = df_full.first()
example

In [ ]:
df_full.count()

In [ ]:
cat = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']


In [ ]:
sel = df_full.limit(10000)
sel.write.csv('/workspace/CreditRiskAI/dev/dev_csv', header = True)